In [1]:
# import libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import nltk # natural language tool kit
# nltk.download('wordnet')
# nltk.download("punkt") # uncomment if package is not already downloaded
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

# create list
words = []
classes = []
documents = []
ignore_letters = ["!", "?", ".", ","] # letters to ignore

intent_file = open("intents.json").read()
intents = json.loads(intent_file)

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        # tokenize each word
        word  = nltk.word_tokenize(pattern)
        words.extend(word) # stores the tokenized word in the list words
        # add to documents
        documents.append((word, intent["tag"])) # combination of word and intent-tag
        # add to classes
        if intent["tag"] not in classes:  # appended only intent["tag"]
            classes.append(intent["tag"])
# print(classes)
# print(documents)
# print(words)
#lemmatize  and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters] #https://www.datacamp.com/community/tutorials/stemming-lemmatization-python
words = sorted(list(set(words)))
# print(words)

# sort the classes
classes = sorted(list(set(classes))) # set helps us to create unique elements

# print(len(documents), "documents")
# print(len(classes), "classes", classes)
# print(len(words), "unique lemmatized word", words)

pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

# create our training data
train = []

#create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    bag_of_words = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag_of_words.append(1) if word in pattern_words else bag_of_words.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =  1
    
    train.append([bag_of_words, output_row])


random.shuffle(train)
train = np.array(train, dtype=object)

# print(train)


Using TensorFlow backend.
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/t

In [3]:
from data_preprocess import *

# Create our train test
train_x = list(train[:,0])
train_y = list(train[:,1])

# Create or model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu")) # First layer with 128 neurons
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu")) #Second layer with 64 neurons
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax")) # Third layer with neurons == no of intents

# Compile model. 
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

# #fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
47/47 [==============================] - 1s 19ms/step - loss: 2.2847 - accuracy: 0.0638
Epoch 2/200
47/47 [==============================] - 0s 1ms/step - loss: 2.1638 - accuracy: 0.1064
Epoch 3/200
47/47 [==============================] - 0s 1ms/step - loss: 2.1227 - accuracy: 0.1489
Epoch 4/200
47/47 [==============================] - 0s 1ms/step - loss: 2.0539 - accuracy: 0.3404
Epoch 5/200
47/47 [==============================] - 0s 847us/step - loss: 1.9915 - accuracy: 0.2553
Epoch 6/200
47/47 [==============================] - 0s 904us/step - loss: 1.8213 - accuracy: 0.4043
Epoch 7/200
47/47 [==============================] - 0s 1ms/step - loss: 1.7899 - accuracy: 0.4894
Epoch 8/200
47/47 [==============================] - 0s 810us/step - loss: 1.6052 - accuracy: 0.5745
Epoch 9/200
47/47 [==============================] - 0s 1ms/step - loss: 1.4940 - accuracy: 0.5957
Epoch 10/200
47/47 [==============================] - 0s 1ms/step - loss: 1.4185 - accuracy: 0.5532
E

47/47 [==============================] - 0s 812us/step - loss: 0.0278 - accuracy: 1.0000
Epoch 80/200
47/47 [==============================] - 0s 812us/step - loss: 0.0372 - accuracy: 1.0000
Epoch 81/200
47/47 [==============================] - 0s 704us/step - loss: 0.0894 - accuracy: 0.9574
Epoch 82/200
47/47 [==============================] - 0s 871us/step - loss: 0.0438 - accuracy: 0.9787
Epoch 83/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0150 - accuracy: 1.0000
Epoch 84/200
47/47 [==============================] - 0s 762us/step - loss: 0.0332 - accuracy: 1.0000
Epoch 85/200
47/47 [==============================] - 0s 727us/step - loss: 0.0700 - accuracy: 0.9787
Epoch 86/200
47/47 [==============================] - 0s 776us/step - loss: 0.0430 - accuracy: 1.0000
Epoch 87/200
47/47 [==============================] - 0s 824us/step - loss: 0.0252 - accuracy: 1.0000
Epoch 88/200
47/47 [==============================] - 0s 856us/step - loss: 0.0138 - accuracy: 1.

Epoch 159/200
47/47 [==============================] - 0s 857us/step - loss: 0.0235 - accuracy: 1.0000
Epoch 160/200
47/47 [==============================] - 0s 785us/step - loss: 0.0078 - accuracy: 1.0000
Epoch 161/200
47/47 [==============================] - 0s 725us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 162/200
47/47 [==============================] - 0s 695us/step - loss: 0.0073 - accuracy: 1.0000
Epoch 163/200
47/47 [==============================] - 0s 712us/step - loss: 0.0031 - accuracy: 1.0000
Epoch 164/200
47/47 [==============================] - 0s 688us/step - loss: 0.0246 - accuracy: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 849us/step - loss: 0.0210 - accuracy: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 843us/step - loss: 0.0214 - accuracy: 0.9787
Epoch 167/200
47/47 [==============================] - 0s 951us/step - loss: 0.0379 - accuracy: 0.9787
Epoch 168/200
47/47 [==============================] - 0s 707us/step - lo

In [4]:
#conda install keras
